# 7.c.i Semantyka dystrybucyjna

Musimy uaktualnić moduł gensim, po wykonaniu poniższej komórki proszę zresetować runtime

In [ ]:
!python3 -m pip install gensim==4.0.0b

#### Wykorzystanie gotowego modelu wektorowego

In [ ]:
!wget http://dsmodels.nlp.ipipan.waw.pl/dsmodels/nkjp+wiki-forms-all-100-cbow-hs.txt.gz
!gunzip nkjp+wiki-forms-all-100-cbow-hs.txt.gz

from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format("nkjp+wiki-forms-all-100-cbow-hs.txt")

#### Praca z wektorami słów

In [ ]:
marchew = wv["marchew"]
print(type(marchew), marchew.shape, "\n", marchew)

#### Podobieństwo słów

Podobieństwo jest liczone jako cosinus kąta między wektorami (znormalizowany iloczyn skalarny wektorów).

In [ ]:
print(wv.similarity("marchew", "cebula"))
print(wv.similarity("marchew", "stół"))

#### Ranking podobieństwa

In [ ]:
wv.most_similar("szklanka")

#### Problem antonimii

In [ ]:
wv.most_similar("ładny")

Kobieta ma się do mężczyzny, jak kto do króla?

Arytmetyka wektorów:

    królowa = król - mężczyzna + kobieta

In [ ]:
wv.most_similar(["król", "kobieta"], negative=["mężczyzna"])

In [ ]:
wv.doesnt_match(["topola", "buk", "bóbr", "sosna"])

### Tworzenie modelu word2vec

#### Przygotowanie danych
Jako zbiór danych wykorzystamy trylogię Sienkiewicza

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/word2vec_data.zip?raw=true
!mv word2vec_data.zip?raw=true word2vec_data.zip
!unzip word2vec_data.zip

In [ ]:
import os
import re

tokenizer = re.compile(r"[\w]+")

def preprocess(doc):
    lowered = doc.lower()
    tokenized = tokenizer.findall(lowered)
    return tokenized

train_docs = []
folder = "word2vec_data"
for file in os.listdir(folder):
    fullpath = os.path.join(folder, file)
    with open(fullpath) as f:
        txt = f.read()
    docs = [preprocess(doc) for doc in txt.split("\n") if doc!=""]
    train_docs.extend(docs)

print(len(train_docs))

#### Inicjalizacja i trening modelu

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=train_docs, vector_size=100, window=3, min_count=2, seed=42, workers=1)
model.train(train_docs, total_examples=len(train_docs), epochs=10)

#### Predykcja słów na podstawie kontekstu

In [ ]:
model.predict_output_word(["z", "rany", "polała", "się"])

### Ewaluacja zanurzeń/embeddingów

#### Wynikowe wektory słów

In [ ]:
wv = model.wv
rzeczpospolita = wv["rzeczpospolita"]
print(type(rzeczpospolita), rzeczpospolita.shape, "\n", rzeczpospolita)

#### Podobieństwo między słowami
Podobieństwo jest liczone jako cosinus kąta między wektorami (znormalizowany iloczyn skalarny wektorów).

In [ ]:
import numpy



man, woman = wv["mężczyzna"], wv["kobieta"]
lena, lenb = numpy.linalg.norm(man), numpy.linalg.norm(woman)
similarity = numpy.dot(man, woman.T)/(lena * lenb)

print(similarity)
print(wv.similarity("kobieta", "mężczyzna"))

#### Ranking podobieństwa dla słowa "kmicic"

In [ ]:
wv.most_similar("kmicic")

#### Problem antonimii

In [ ]:
wv.most_similar("pogodny")

#### Odnajdywanie analogii
Oleńka ma się do Kmicica, jak kto do Wołodyjowskiego?

Arytmetyka wektorów:

    basia = oleńka - kmicic + wołodyjowski

In [ ]:
wv.most_similar(["oleńka", "wołodyjowski"], negative=["kmicic"])

#### Wykrywanie elementów niepasujących do reszty

In [ ]:
wv.doesnt_match(["szabla", "armata", "muszkiet", "oko"])

### Wizualizacja zanurzeń

#### Redukcja wymiarowości

In [ ]:
from sklearn.decomposition import PCA

dim_reductor = PCA(n_components=2)
words = ["noc", "dzień", "kobieta", "mężczyzna", "pies", "koń", "kot", "łzy", "krew"]

data = [wv[w] for w in words]

reduced = dim_reductor.fit_transform(data)
print(reduced)

#### Rysowanie wykresu

In [ ]:
import matplotlib.pyplot as plt


x = [p[0] for p in reduced]
y = [p[1] for p in reduced]

fig, ax = plt.subplots()
ax.scatter(x, y)

for i, txt in enumerate(words):
    ax.annotate(txt, (x[i], y[i]))


#fig.show()

Dla pełniejszego wglądu, warto skorzystać z narzędzia https://projector.tensorflow.org, poniżej zapisywanie wektorów w wymaganym formacie.

In [ ]:
labels = [wv.index_to_key[i] for i in range(len(wv))]
lab_tsv = "\n".join(labels)
vec_tsv = "\n".join(["\t".join([str(v) for v in wv[lab]]) for lab in labels])
with open("lab.tsv", "w") as f:
    f.write(lab_tsv)
with open("vec.tsv", "w") as f:
    f.write(vec_tsv)

### Zapisywanie i wczytywanie modelu

In [ ]:
from gensim.models import KeyedVectors

wv.save_word2vec_format("trylogia.vec")
wv2 = KeyedVectors.load_word2vec_format("trylogia.vec")